# Comparison of Florianopolis Geometry
Multiple climate zones, comparison of CS 5-zone, umi and surrogate

In [1]:
# import plotly
import plotly.express as px
import pandas as pd
import numpy as np
import os
from pathlib import Path
from geomeppy.patches import EpBunch
from utils.schedules import get_schedules
from utils.constants import *
import json
from archetypal import IDF, BuildingTemplate
from archetypal.umi_template import UmiTemplateLibrary
from umi.ubem import init_template

ref_buildings_path = "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Building Energy References/ReferenceBuildings"
results_dir = "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Florianopolis/CS Sensitivity"

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.8.16
[Taichi] Starting on arch=cuda


In [2]:
month_map = {
    "january": 744,
    "february": 672,
    "march": 744,
    "april": 720,
    "may": 744,
    "june": 720,
    "july": 744,
    "august": 744,
    "september": 720,
    "october": 744,
    "november": 720,
    "december": 744,
}
sum(list(month_map.values()))

8760

## Umi results

In [24]:
# Import umi results for 4A (boston)
dfs = []
for load in ["heating", "cooling"]:
    df = pd.read_csv(Path(results_dir, f"UMI_florianopolis_test_bos_{load}.csv"))
    df["load"] = load
    dfs.append(df)
umi_df = pd.concat(dfs)

In [25]:
umi_df["annual"] = umi_df[[str(x) for x in range(1, 8761)]].sum(axis=1)

In [26]:
umi_df.head()

,Building Id,Building Name,Metric,Units,Resolution,1,2,3,4,5,...,8753,8754,8755,8756,8757,8758,8759,8760,load,annual
0,03b1d5a3-ebbf-4560-a399-b81bc391f9f7,03b1d5a3-ebbf-4560-a399-b81bc391f9f7,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,8803.872406
1,06e706f2-dcac-4c99-a4da-0afbdba2aa9a,06e706f2-dcac-4c99-a4da-0afbdba2aa9a,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,32649.884940
2,0b48b5a6-e91b-4fc9-aeaf-5805a420a8e1,0b48b5a6-e91b-4fc9-aeaf-5805a420a8e1,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,2338.561208
3,0dabd728-d95f-40d5-8836-eba55eba02c2,0dabd728-d95f-40d5-8836-eba55eba02c2,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,2219.940068
4,0e188b41-61da-4d8d-aaf3-be4502ac3cb7,0e188b41-61da-4d8d-aaf3-be4502ac3cb7,SDL/Heating,kWh,Hourly,0,0,0,0,0,...,0,0,0,0,0,0,0,0,heating,4282.442673


In [113]:
def get_monthly(df):
    h = df.copy()
    h.set_index(["Building Id", "load"], inplace=True)
    nums = h[[str(x) for x in range(1, 8761)]]
    nums.columns = nums.columns.astype("int")
    c = 0
    print(nums.shape)
    for month, hours in month_map.items():
        h[month] = nums.iloc[:, c : c + hours].sum(axis=1)
        c += hours
    return h[list(month_map.keys())]

In [87]:
umi_all_monthly = get_monthly(umi_df).groupby("load").sum()
px.bar(umi_all_monthly.T, x=umi_all_monthly.T.index, y=["cooling", "heating"], title="Long-Form Input")

## CS Results (with shading)

In [166]:
# CS with shading
with open(Path(results_dir, "results-out-boston.json"), "r") as f:
    CS_results = json.load(f)
CS_results = pd.DataFrame.from_dict(CS_results)

In [98]:
# CS individual buildings
CS_building_results = {}
for i in range(93):
    with open(Path(results_dir) / f"SingleBuildings_bostonresults-out-{i}.json", "r") as f:
        r = json.load(f)
        res_dict = {}
        for l in ["heating", "cooling"]:
            res_dict.update({(l, k): r for k, r in zip(month_map.keys(), r[l])})
        CS_building_results[i] = res_dict
CS_building_results = pd.DataFrame.from_dict(CS_building_results).T
CS_building_results.head()

heating                                                                 \
     january   february      march      april       may      june      july   
0  32.090668  23.893605  19.669354   7.891526  2.921961  0.908931  0.088472   
1  44.966179  32.842645  26.579238   9.268430  2.754648  0.662748  0.015629   
2  44.320260  32.257416  25.943443   8.540037  2.273449  0.520068  0.008804   
3  73.664598  55.341310  46.160150  17.701076  5.917686  1.467348  0.058449   
4  15.747004  11.174053   7.954368   1.724187  0.129764  0.028062  0.000015   

                                  ...   cooling                      \
     august september    october  ...     march     april       may   
0  0.157210  1.140951   6.404070  ...  1.549111  2.451621  4.482587   
1  0.020090  0.775616   7.235908  ...  1.861321  3.382152  6.571529   
2  0.010966  0.582833   6.500505  ...  1.704264  3.234011  6.448061   
3  0.056138  1.625255  13.780964  ...  1.901292  3.317164  6.807507   
4  0.000006  0.014929   0.866680  ...  0.418849  1.156582  2.681583   

                                                                            
        june       july     august september   october  november  december  
0   7.004161   8.785800   8.721167  5.314509  2.887471  2.171425  0.919858  
1  10.671434  13.702258  13.562467  8.023152  4.000234  2.710544  0.984728  
2  10.685268  13.892239  13.765921  8.119133  3.964886  2.628734  0.896105  
3  12.046817  15.937187  15.599665  8.903555  4.150727  2.901337  1.043165  
4   4.799353   6.448598   6.358716  3.840722  1.618487  0.815659  0.221070  

[5 rows x 24 columns]

## Model results

In [149]:
COP_COOLING = 3.32337
COP_HEATING = 0.75

In [100]:
import geopandas as gpd
from ladybug.epw import EPW

gdf = gpd.read_file(
    Path("D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Florianopolis")
    / "Florianopolis_Baseline.zip"
)

# dict to store key fields for known gis files
id_cols = {
    "florianpolis": {
        "height_col": "HEIGHT",
        "id_col": "OBJECTID",
        "template_name_col": "template_name",
        "wwr_col": "wwr",
    }
}

# epw_fp = (
#     Path(
#         "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Building Energy References/ReferenceBuildings"
#     )
#     / "USA_FL-MIAMI.722020_TMY2-12839.epw"
# )
epw_fp = ("C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2004-2018.epw")


epw = EPW(epw_fp)
template_lib = UmiTemplateLibrary.open(
    Path(
        "D:/Users/zoelh/Dropbox (MIT)/Thesis Work/Building Energy References/ArchetypalReferences"
    )
    / "DefaultTemplateLibrary_1A.json"
)

# Insert dummy template names
gdf[id_cols["florianpolis"]["template_name_col"]] = np.random.choice(
    [t.Name for t in template_lib.BuildingTemplates], size=gdf.shape[0]
)
# insert dummy wwrs
gdf[id_cols["florianpolis"]["wwr_col"]] = 0.4

In [101]:
from umi.ubem import UBEM

ubem = UBEM(
    gdf=gdf,
    epw=epw,
    **id_cols["florianpolis"],
    template_lib=template_lib,
    sensor_spacing=3,
    shoebox_width=3,
    floor_to_floor_height=3,
    perim_offset=3,
    shoebox_gen_type="edge_unshaded",
)

DEBUG:UMI:INITIAL GEOMETRY CHECK: 0 NaNs, 0 open polygons
DEBUG:UMI:Fetching BuildingTemplate vector data from RefBldgFullServiceRestaurantNew2004_v1.4_7.2_1A_USA_FL_MIAMI
DEBUG:UMI:Found facade with r_value 2.6 and tmass bin 3
DEBUG:UMI:Found slab with r_value 0.08
DEBUG:UMI:Found roof with r_value 0.06 and tmass bin 3
DEBUG:UMI:Using internal shgc calculation.
DEBUG:UMI:Thermal mass roof 3, walls 3.
DEBUG:UMI:Fetching BuildingTemplate vector data from RefBldgHospitalNew2004_v1.4_7.2_1A_USA_FL_MIAMI
DEBUG:UMI:Found facade with r_value 3.44 and tmass bin 2
DEBUG:UMI:Found slab with r_value 0.08
DEBUG:UMI:Found roof with r_value 3.53 and tmass bin 2
DEBUG:UMI:Using internal shgc calculation.
DEBUG:UMI:Thermal mass roof 2, walls 2.
DEBUG:UMI:Fetching BuildingTemplate vector data from RefBldgLargeHotelNew2004_v1.4_7.2_1A_USA_FL_MIAMI
DEBUG:UMI:Found facade with r_value 2.18 and tmass bin 2
DEBUG:UMI:Found slab with r_value 0.28
DEBUG:UMI:Found roof with r_value 3.53 and tmass bin 2
DEBUG:

In [102]:
# ubem.prepare_epw_features(epw=EPW("C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2004-2018.epw"))
features, schedules, climate = ubem.prepare_for_surrogate()
print("buildings", len(ubem.gis_features_df), "sbs", len(features))

buildings 93 sbs 1022


In [103]:
from ml.surrogate import Surrogate
from ml.predict import predict_ubem
from lightning.pytorch import Trainer
import torch

bucket = "ml-for-bem"
remote_experiment = "full_climate_zone/v7"
local_data_dir = "data/lightning"

registry = "ml-for-building-energy-modeling/model-registry"
model = "Global UBEM Shoebox Surrogate with Combined TS Embedder"
tag = "v3"
resource = "model.ckpt"

surrogate = Surrogate.load_from_registry(registry, model, tag, resource)
surrogate.model.eval()

trainer = Trainer(
    accelerator="auto",
    devices="auto",
    strategy="auto",
    enable_progress_bar=True,
)

Couldn't import dot_parser, loading of dot files will not be possible.


c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'target_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['target_transform'])`.
c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'weather_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['weather_transform'])`.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, u

In [104]:
torch.set_float32_matmul_precision("medium")
shoebox_predictions, monthly_predictions, annual_predictions = predict_ubem(
    trainer=trainer,
    surrogate=surrogate,
    features=features,
    schedules=schedules,
    climate=climate,
    batch_size=32,
    apply_cops=False,
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   6%|▋         | 2/32 [00:00<00:06,  4.76it/s]

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning:

Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Convolution.cpp:1009.)



Predicting DataLoader 0: 100%|██████████| 32/32 [00:02<00:00, 15.37it/s]


In [105]:
annual_predictions.describe()

Name,Cooling,Heating
count,93.000000,93.000000
mean,160.484887,161.439446
std,56.036485,88.971107
min,55.313561,40.286324
25%,123.569832,90.665610
50%,159.319997,144.489333
75%,198.060692,217.129912
max,274.791931,479.999588


## Compare

In [108]:
tfa = ubem.gis_features_df.bldg_footprint_area * ubem.gis_features_df.floor_count

In [110]:
# px.bar(x=annual_predictions.index, y=annual_predictions["Cooling"], title="Surrogate")
fig = px.bar(
    x=annual_predictions.index,
    y=[
        annual_predictions.Cooling.divide(COP_COOLING),
        umi_df[umi_df["load"] == "cooling"]["annual"].divide(tfa, axis=0),
        CS_building_results['cooling'].sum(axis=1),
    ],
    title="Florianopolis (1A) Cooling with COP: UMI (red), surrogate (blue)",
    barmode="group"
)
fig.show()

In [150]:

fig = px.bar(
    x=annual_predictions.index,
    y=[
        annual_predictions.Heating.divide(COP_HEATING),
        umi_df[umi_df["load"] == "heating"]["annual"].divide(tfa, axis=0),
        CS_building_results['heating'].sum(axis=1),
    ],
    title="Florianopolis (1A) Heating with COP: UMI (red), surrogate (blue)",
    barmode="group"
)
fig.show()

In [145]:
umi_df_mon_norm = get_monthly(umi_df).groupby("load").sum().divide(tfa.sum())
umi_df_mon_norm

(186, 8760)


,january,february,march,april,may,june,july,august,september,october,november,december
load,,,,,,,,,,,,
cooling,0.000000,0.00000,0.000000,0.000000,0.0,0.720345,2.467,3.391396,2.156917,0.969094,0.645356,0.395199
heating,1.176728,5.15668,3.361244,0.533238,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000


In [183]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = monthly_predictions["Cooling"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_COOLING),
        mode='lines+markers',
        name = "Surrogate, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = monthly_predictions["Heating"].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()*COP_HEATING),
        mode='lines+markers',
        name = "Surrogate, Heating"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_results["cooling"],
        mode='lines+markers',
        name = "CS context, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_results["heating"],
        mode='lines+markers',
        name = "CS context, Heating"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_building_results['cooling'].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_building_results['heating'].multiply(tfa, axis=0).sum(axis=0).divide(tfa.sum()),
        mode='lines+markers',
        name = "CS no-shad, Heating"
    )
)

# colors = []
# for i in range(4):
#     fig.data[i].line.color = colors[i]
fig.update_layout(
    title="Surrogate, CS context, and CS-no shading average normalized heating and cooling",
    autosize=False,
    width=800,
    height=600,
)
fig.show()

In [177]:

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = monthly_predictions["Cooling"].multiply(tfa, axis=0).sum(axis=0).divide(COP_COOLING),
        mode='lines+markers',
        name = "Surrogate, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = monthly_predictions["Heating"].multiply(tfa, axis=0).sum(axis=0).divide(COP_HEATING),
        mode='lines+markers',
        name = "Surrogate, Heating"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_results["cooling"].multiply(tfa.sum()),
        mode='lines+markers',
        name = "CS context, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_results["heating"].multiply(tfa.sum()),
        mode='lines+markers',
        name = "CS context, Heating"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_building_results['cooling'].multiply(tfa, axis=0).sum(axis=0),
        mode='lines+markers',
        name = "CS context, Cooling"
    )
)
fig.add_trace(
    go.Scatter(
        x = umi_df_mon_norm.columns,
        y = CS_building_results['heating'].multiply(tfa, axis=0).sum(axis=0),
        mode='lines+markers',
        name = "CS context, Heating"
    )
)

# colors = []
# for i in range(4):
#     fig.data[i].line.color = colors[i]
fig.update_layout(
    title="Surrogate, ClimateStudio-shading, and ClimateStudio-no shading total heating and cooling"
)
fig.show()